In [ ]:
import pandas as pd
import re

df = pd.read_csv('data/reality_df.csv')
# dropnout zbytecne sloupce
cols_to_drop = ['title', 'obcanska_vybavenost', 'id', 'available_from', 'updated', 'address']
df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# vytvorit mapu slov, kde se snazim cislovku slovy prevest na cislo (integer)
num_wordings = ['první', 'druhé', 'třetí', 'čtvrté', 'páté', 'šesté', 'sedmé', 'osmé', 'deváté', 'desáté',
                'jedenácté', 'dvanácté', 'třinácté', 'čtrnácté', 'patnácté', 'šestnácté', 'sedmnácté', 'osmnácté',
                'devatenácté']
# mapping
mapping = {'první': 1, 'druhé': 2, 'třetí': 3, 'čtvrté': 4, 'páté': 5, 'šesté': 6, 'sedmé': 7, 'osmé': 8,
           'deváté': 9, 'desáté': 10, 'jedenácté': 11, 'dvanácté': 12, 'třinácté': 13, 'čtrnácté': 14,
           'patnácté': 15, 'šestnácté': 16, 'sedmnácté': 17, 'osmnácté': 18, 'devatenácté': 19}

# funkce, co vyhodi bud 0 = prizemi, last = posledni patro, nebo cislovku patra z description
def get_floor(row):
  if 'poslední' in str(row).strip():
    patro = 'last'
  elif 'přízem' in str(row).strip():
    patro = '0'
  else:
    # hleda slovo "patr" a pred nimi cislovku nebo slovo s cislovkou
    patro = re.search(r'([\w]+)(\s)([\w.]+)(\s)(pat[řero]+)', row)

    # pokud regex naleze pattern, tedy slovo patro
    if patro is not None:

      # vlozit nalezeny patter do promenne
      patro = patro[0]

      # najit cislovku (integer)
      number = re.search(r'\d', patro)

      # pokud nalezneme cislovku, pak cislovka = patro
      if number is not None:
        patro = number[0]

      # pokud nenalezneme cislovku, pak
      else:

        # projet loopem slova z mappingu, tedy cislovky psany slovy
        for word in num_wordings:
          if word in patro.strip():
            patro = mapping[word]
            break
  return patro

# aplikovat funkci vyse, vyplnit tedy missing values tim, co vyplivla
df['floor'].fillna(df['description'].apply(get_floor), inplace=True)


In [ ]:
# pokud je v description zmineno slovo "sklep", pak vyhodi true, jinak vyhodi None (ne False, protoze
# neni jistota, ze absence zminky o sklepu znamena absenci sklepu)

def get_basement_from_description(row):
  return True if re.search(r'(sklep)+', row) is not None else None

# nahradit missing values v basement tam, kde to lze vycist z desc
df['basement'].fillna(df['description'].apply(get_basement_from_description), inplace = True)

In [ ]:
# pokud je v description zmineno slovo "garaz" nebo "garáž", pak vyhodi true, jinak vyhodi None (ne False, protoze
# neni jistota, ze absence zminky o garáže znamena absenci garáže)

def get_garage_from_description(row):
  return True if re.search(r'(garaz|garáž)+', row) is not None else None

# nahradit missing values v basement tam, kde to lze vycist z desc
df['garage'].fillna(df['description'].apply(get_garage_from_description), inplace = True)

In [ ]:
# pokud je v description zmineno slovo "výtah" nebo "vytah", pak vyhodi true, jinak vyhodi None (ne False, protoze
# neni jistota, ze absence zminky o garáže znamena absenci garáže)

def get_elevator_from_description(row):
  elevator = True if re.search(r'(vytah|výtah)+', row) is not None else None
  return elevator

# nahradit missing values v basement tam, kde to lze vycist z desc
df['elevator'].fillna(df['description'].apply(get_elevator_from_description), inplace = True)

In [ ]:
#naplnit PENB, protoze neuvedene PENB (tedy nezname, neudelane) ze zakona musi byt oznaceno jako G, tedy nejhorsi

df['penb'].fillna('G', inplace = True)

# TODO: build some kickass model